# 计算智能练习2

**运行环境：**

In [114]:
import numpy as np
import os
import matplotlib.pyplot as plt
from struct import unpack
from PIL import Image

## 1. 数据

文件夹orl中包含40个人的人脸图像，每个人有10幅图像，共400幅图像。

### 1.1 数据读取

In [125]:
img_data_path = "./data-ORL/ORL/"
img_names = os.listdir(img_data_path)
img_names.remove("Thumbs.db")
imgs = [img_data_path + img_name for img_name in img_names]
#print(imgs)

img_array_list = []
for img in imgs:
    im = Image.open(img)
    img_array = np.array(im).flatten()
    img_array_list.append(img_array)

#print(img_array_list)

img_dataset = np.asarray(img_array_list)

print("样本个数：%d" %len(img_dataset))
print("样本示例：\n", img_dataset)
print("数据集大小： ", img_dataset.shape)

样本个数：400
样本示例：
 [[ 48  49  45 ...  47  46  46]
 [ 60  60  62 ...  32  34  34]
 [ 39  44  53 ...  29  26  29]
 ...
 [125 119 124 ...  36  39  40]
 [119 120 120 ...  89  94  85]
 [125 124 124 ...  36  35  34]]
数据集大小：  (400, 10304)


### 1.2 数据切分

按照要求将数据集划分为**训练集**和**测试集**，实验要求每个人的前5个人脸为训练集，后5个人脸为测试集。

In [156]:
step = 10
half_step = step//2

train_X = img_dataset[0 : half_step, :]
test_X  = img_dataset[half_step : step, :]
labels  = np.zeros((5,1), dtype=np.int)

l = 1
for i in range(step, len(img_dataset), step):
    #print("range(%d, %d)" %(i, i+step))
    train_X = np.concatenate([train_X, img_dataset[i : i+half_step, :]], axis=0)
    #print("range(%d, %d)" %(i, i+i+half_step))
    test_X  = np.concatenate([test_X, img_dataset[i+half_step : i+step, :]], axis=0)
    #print("range(%d, %d)" %(i + half_step, i+step))
    #生成测试集标签
    labels = np.concatenate([labels, l*np.ones((5,1), dtype=np.int)], axis=0)
    l += 1

print("训练集 train_X 大小: ", train_X.shape)
print("测试集 test_X 大小: ", test_X.shape)
print("labels :", labels.shape)

训练集 train_X 大小:  (200, 10304)
测试集 test_X 大小:  (200, 10304)
labels : (200, 1)


## 2. 最小距离分类器



### 2.1 计算特征中心

这里每一维度的量纲相同，因此无需进行归一化

In [137]:
mu_list = []
for i in range(0, len(train_X), 5):
    #print("range(%d, %d)" %(i, i+5))
    smaple = train_X[i:i+5, :]
    mu = np.mean(smaple, axis=0)
    mu_list.append(mu)

#各类中心
mu_centers = np.asarray(mu_list)
print(mu_centers)
print(mu_centers.shape)

[[ 54.8  56.4  55.  ...  36.8  30.6  34.4]
 [ 34.2  35.2  35.6 ...  50.6  51.   51.4]
 [102.  103.2 105.2 ...  43.6  42.2  42.4]
 ...
 [106.2 109.4 109.  ...  66.6  63.6  62. ]
 [ 78.6  80.8  79.4 ... 124.2 121.8 122. ]
 [127.  125.  125.8 ...  67.4  68.8  68. ]]
(40, 10304)


### 2.2 最小距离分类

计算每个样本到各个特征中心的欧氏距离

In [165]:
labels_ = np.zeros((200, 1), dtype=np.int)
for i in range(len(test_X)):
    distances = np.sum((mu_centers - test_X[i,:])**2, axis=1)
    labels_[i] = np.argmin(distances)

#print(labels_)
#print(labels - labels_)
ans = np.count_nonzero(labels - labels_)
print(ans)
print("分类错误率： %f" %(ans/len(labels)))

30
分类错误率： 0.150000
